# Snowflake Trail for Observability
[Snowflake Trail](https://www.snowflake.com/en/data-cloud/snowflake-trail/) is a set of Snowflake capabilities that enables developers to better monitor, troubleshoot, debug, and take actions on pipelines, applications, user code, and compute utilization.

## Truck Analysis
In this demo, we'll explore how to add observability - traces, logs, and alerts for a simple Truck Reviews sentiment analysis use case. We'll integrate [Slack Webhook](https://api.slack.com/messaging/webhooks) to deliver notifications to a Slack channel.

By the end of this demo, you will understand:
- How to enable Telemetry in your Snowflake account
- What the various object levels are at which Telemetry can be set
- How to define Serverless Alerts
- How to integrate Slack notifications via Webhooks

>**IMPORTANT**
>
>Before getting started, make sure you have access to a Slack workspace where you can add a webhook integration


In [ ]:
SELECT current_role() as current_role

In [ ]:
import streamlit as st

## Object Names
Let us define variables that will hold the various object and resource names used throughout this demo.

In [ ]:
__current_role=sql_currents.to_pandas().iloc[0]['CURRENT_ROLE']
__current_role
__database = "kamesh_build_24_demos"
__analytics_schema = "analytics"
__data_schema = "data"
__stages_schema = "stages"
__src_schema = "src"
__task_schema = "tasks"
__alerts_schema = "alerts_and_notifications"
__telemetry_schema = "telemetry"
__warehouse = "kamesh_snowpark_demo_wh"
__task_name = "truck_sentiment"

## Database Setup
In the following steps, we will:
- Create necessary Snowflake objects and resources
- Ingest data required for truck sentiment analysis
- Set up alert triggers for Slack channel notifications

In [ ]:
CREATE DATABASE IF NOT EXISTS {{__database}};
USE DATABASE {{__database}};

Let us create schemas to group our various objects.

| Schema Name | Purpose |
| :----: | :---- |
| analytics | Holds the analytical data |
| stages | Holds all internal and external stages |
| src | Holds the sources of the UDFs and Stored Procedures |
| task | Holds all Tasks used in this demo |
| alerts | Holds all Alert definitions |
| telemetry | Holds database-level event table |


In [ ]:
CREATE SCHEMA IF NOT EXISTS {{__analytics_schema}};
CREATE SCHEMA IF NOT EXISTS {{__data_schema}};
CREATE SCHEMA IF NOT EXISTS {{__stages_schema}};
CREATE SCHEMA IF NOT EXISTS {{__src_schema}};
CREATE SCHEMA IF NOT EXISTS {{__task_schema}};
CREATE SCHEMA IF NOT EXISTS {{__alerts_schema}};
CREATE SCHEMA IF NOT EXISTS {{__telemetry_schema}};

SHOW SCHEMAS in database {{__database}};

## Load Truck Data
The demo uses truck data from Tasty Bytes. Please ensure that you load the data from `data_load.sql` script into your `__database`. The SQL objects and other related data definitions are available [here](https://github.com/Snowflake-Labs/build24-trail-demo/tree/main/scripts).

In [ ]:
CREATE API INTEGRATION if not exists git_api_integration
  API_PROVIDER = git_https_api
  API_ALLOWED_PREFIXES = ('https://github.com/snowflake-labs')
  ENABLED = TRUE;

CREATE OR REPLACE GIT REPOSITORY {{__database}}.{{__data_schema}}.build24_trail_demo
  API_INTEGRATION = git_api_integration
  ORIGIN = 'https://github.com/snowflake-labs/build24-trail-demo';

Let's refresh the repository and pull the latest changes.

In [ ]:
ALTER git repository {{__database}}.{{__data_schema}}.build24_trail_demo fetch;
ls @{{__database}}.{{__data_schema}}.build24_trail_demo/branches/main/scripts;

Let's run the script to create the database objects and ingest the data.

In [ ]:
EXECUTE IMMEDIATE FROM @{{__database}}.{{__data_schema}}.build24_trail_demo/branches/main/scripts/data_setup.j2.sql 
USING ( demo_role => '{{__current_role}}', demo_database => '{{__database}}' );

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.core import CreateMode, Root
from snowflake.core.schema import Schema
from snowflake.core.database import Database

session = get_active_session()
root = Root(session)


## UDF Sentiment Class
A Python UDF that converts Snowflake Cortex sentiment scores into textual sentiment classifications: `negative`, `neutral`, or `positive`.

In [ ]:
from snowflake.core.stage import Stage

__udf_stage_name = "udfs"
__udf_stage = Stage(name=__udf_stage_name)
_ = root.databases[__database].schemas[__src_schema].stages.create(
    __udf_stage,
    mode=CreateMode.if_not_exists,
)

In [ ]:
from snowflake.snowpark.functions import udf

@udf(
    name=f"{__database}.{__data_schema}.classify_sentiment",
    is_permanent=True,
    packages=["snowflake-telemetry-python"],
    stage_location=f"{__database}.{__src_schema}.{__udf_stage_name}",
    replace=True,
)
def classify_sentiment(sentiment_score: float) -> str:
    """Classify sentiment as positive,neutral or negative based on the score."""
    import logging

    import snowflake.telemetry as telemetry

    logging.info("Classifying sentiment score")

    telemetry.set_span_attribute("processing", "classify_sentiment")
    logging.debug(f"Classifying sentiment score {sentiment_score:.2f}")

    if sentiment_score < -0.5:
        logging.debug(f"Sentiment {sentiment_score:.2f} is negative")
        return "negative"
    elif sentiment_score >= 0.5 and sentiment_score <= 1.0:
        logging.debug(f"Sentiment {sentiment_score:.2f} is positive")
        return "positive"
    else:
        logging.debug(f"Sentiment {sentiment_score:.2f} is neutral")
        return "netural"

## Stored Procedure `truck_review_sentiments`
The stored procedure builds the `truck_review_sentiments` table and uses the `sentiment_class` UDF to categorize sentiments into text classifications.

In [ ]:
# stage to hold the stored procedure sources
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.session import Session
from snowflake.core.stage import Stage

__pros_stage_name = "procs"
__procs_stage = Stage(name=__pros_stage_name)
_ = (
    root.databases[__database]
    .schemas[__src_schema]
    .stages.create(
        __procs_stage,
        mode=CreateMode.if_not_exists,
    )
)

@sproc(
    name=f"{__database}.{__data_schema}.build_truck_review_sentiments",
    replace=True,
    is_permanent=True,
    packages=[
        "snowflake-telemetry-python",
        "snowflake-ml-python",
    ],
    stage_location=f"{__database}.{__src_schema}.{__procs_stage.name}",
    source_code_display=True,
    comment="Build the build_truck_review_sentiments table. This procedure will be called from a Task.",
)
def build_truck_review_sentiments(session: Session) -> None:
    """Build the Truck Review Sentiments table."""
    import logging

    import snowflake.cortex as cortex
    import snowflake.snowpark.functions as F
    import snowflake.telemetry as telemetry
    from snowflake.snowpark.types import DecimalType

    logging.debug("START::Truck Review Sentiments")
    telemetry.set_span_attribute("executing", "build_truck_review_sentiments")

    try:
        telemetry.set_span_attribute("building", "truck_reviews")
        review_df = (
            session.table(f"{__database}.{__analytics_schema}.truck_reviews_v")
            .select(
                F.col("TRUCK_ID"),
                F.col("REVIEW"),
            )
            .filter(F.year(F.col("DATE")) == 2024)
        )
        telemetry.set_span_attribute("building", "add_sentiment_score")
        review_sentiment_score_df = review_df.withColumn(
            "SENTIMENT_SCORE",
            cortex.Sentiment(F.col("REVIEW")).cast(DecimalType(2, 2)),
        )
        telemetry.set_span_attribute("building", "add_sentiment_class")
        review_sentiment_class_df = review_sentiment_score_df.withColumn(
            "SENTIMENT_CLASS",
            classify_sentiment(
                F.col("SENTIMENT_SCORE"),
            ),
        )
        logging.debug(review_sentiment_score_df.show(5))
        __table = f"{__database}.{__data_schema}.truck_review_sentiments"
        telemetry.set_span_attribute("save", f"save_to_{__table}")
        review_sentiment_class_df.write.mode("overwrite").save_as_table(__table)
    except Exception as e:
        logging.error(f"Error building truck_review_sentiments,{e}", exc_info=True)

    logging.debug("END::Truck Review Sentiments Complete")


## Telemetry Settings
In the following steps, we will set up Telemetry Events (logs/traces) at the database level. While Snowflake defaults to storing events in `SNOWFLAKE.TELEMETRY.EVENTS`, for this demo we will configure event collection at the database level.

In [ ]:
-- check the current event_table
SHOW PARAMETERS LIKE 'event_table' IN DATABASE {{__database}};

Create the event table at the database level and set it as the default Events table for the database.

In [ ]:
-- create event table 
CREATE EVENT TABLE IF NOT EXISTS {{__database}}.{{__telemetry_schema}}.events;
-- set to new event table
ALTER DATABASE {{__database}} SET EVENT_TABLE = {{__database}}.{{__telemetry_schema}}.events;

In the following cells, we will examine the parameters for logs, traces, and metrics in the demo database.

In [ ]:
SHOW PARAMETERS LIKE 'LOG_LEVEL' IN DATABASE {{__database}};

In [ ]:
SHOW PARAMETERS LIKE 'TRACE_LEVEL' IN DATABASE {{__database}};

In [ ]:
SHOW PARAMETERS LIKE 'METRIC_LEVEL' IN DATABASE {{__database}};

Alter the demo database to set the logging level to DEBUG, trace level to ALWAYS, and metrics collection level to ALL

In [ ]:
-- set log, trace and metrtic levels
ALTER DATABASE {{__database}} SET LOG_LEVEL = DEBUG;
ALTER DATABASE {{__database}} SET TRACE_LEVEL = ALWAYS;
ALTER DATABASE {{__database}} SET METRIC_LEVEL = ALL;

## Truck Reviews
Let's ensure we have the data ingested and ready to use.

In [ ]:
select * 
from {{__database}}.analytics.truck_reviews_v
limit 5;

## Tasks
Let's create a few tasks to execute the stored procedure and build our truck_review_sentiments table.

In [ ]:
from datetime import timedelta

from snowflake.core.task import StoredProcedureCall, Task

truck_sentiment_task = Task(
    name=__task_name,
    warehouse=__warehouse,
    definition=StoredProcedureCall(build_truck_review_sentiments),
    schedule=timedelta(minutes=1),
)

task_truck_sentiment = (
    root.databases[__database].schemas[__task_schema].tasks[__task_name]
)

task_truck_sentiment.create_or_alter(truck_sentiment_task)

In [ ]:
tasks = root.databases[__database].schemas[__task_schema].tasks
__task_truck_sentiment = tasks[__task_name]
task_detials = __task_truck_sentiment.fetch()
st.write(f"Current Task Status:`{task_detials.state}`")

Resume the task.

In [ ]:
__task_truck_sentiment.resume()

Suspend the task.

In [ ]:
__task_truck_sentiment.suspend()

Execute the task immediately.

In [ ]:
__task_truck_sentiment.execute()

# Alerts and Notifications

## Serverless Alerts
Alerts that use the serverless compute model are called serverless alerts. When using the serverless compute model, Snowflake automatically resizes and scales the required compute resources for the alert. Snowflake determines the ideal compute resource size for each run based on a dynamic analysis of statistics from the alert's most recent previous executions.

## Slack Notifications
To create a Slack Webhook notification, we need to complete the following steps:

1. Create a Slack Webhook using the [Slack API](https://api.slack.com/apps) to enable posting to a channel. For detailed instructions, refer to the [Slack Webhooks documentation](https://api.slack.com/messaging/webhooks).

2. Obtain the Slack Webhook URL for channel posting. The URL format follows this pattern:
   `https://hooks.slack.com/services/<slack webhook secret content>`

3. Create a string-type secret containing the `<slack webhook secret content>` value.

4. Create a `NOTIFICATION INTEGRATION` using both the `secret` and the `Slack Webhook URL`.

### Create Slack Webhook Secret
The Slack webhook secret can be extracted from the Webhook URL. For example, if your URL is `https://hooks.slack.com/services/Txxxxxxx/B000000000/xxxxxxxxxx`, use the string `Txxxxxxx/B000000000/xxxxxxxxxx` as the `SECRET_STRING`.

In [ ]:
slack_webhook_secret = st.text_input("Enter Slack Webhook Secret:",type="password")
if slack_webhook_secret == "":
    raise Exception("Slack webhook secret is required.")

Let's define variables to hold the names of the alert and notification objects.

In [ ]:
__slack_webhook_secret_name='slack_alerts_notifications_webhook'
__slack_notification='slack_channel_alerts_notify'
__truck_negatives_alert='truck_review_alert'

Let's create a secret to hold the Slack webhook secret.

In [ ]:
CREATE OR REPLACE SECRET {{__database}}.{{__alerts_schema}}.{{__slack_webhook_secret_name}}
  TYPE = GENERIC_STRING
  SECRET_STRING = '{{slack_webhook_secret}}';

[Notification Integration](https://docs.snowflake.com/en/sql-reference/commands-integration) enables us to trigger a notification on an alert.

In [ ]:
-- send to channel 
CREATE OR REPLACE NOTIFICATION INTEGRATION {{__slack_notification}}
  TYPE = WEBHOOK
  ENABLED = true
  WEBHOOK_URL = 'https://hooks.slack.com/services/SNOWFLAKE_WEBHOOK_SECRET'
  WEBHOOK_SECRET = {{__database}}.{{__alerts_schema}}.{{__slack_webhook_secret_name}}
  WEBHOOK_BODY_TEMPLATE='SNOWFLAKE_WEBHOOK_MESSAGE'
  WEBHOOK_HEADERS=('Content-Type'='application/json');

## Serverless Alert
Let's define a serverless alert that triggers when data in `truck_review_sentiments` has the class `negative` and a sentiment score less than `-0.8`. For simplicity in this demo, we will retrieve only the top three negative records.

Once we have the negative records, we will use [Cortex Complete](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex) to construct a Slack message that will be sent as part of the notification.

> *NOTE*:
>
> To convert a normal alert to a serverless alert, omit the `WAREHOUSE` property.

In [ ]:
-- Alert - alerts when there is stronger negative feedback
-- Truck Review Alert
CREATE OR REPLACE ALERT {{__database}}.{{__alerts_schema}}.{{__truck_negatives_alert}}
  SCHEDULE = '1 minute'
  IF(
      EXISTS(
        WITH negative_reviews AS (
            SELECT 
                truck_id,
                review,
                sentiment_score,
                ROW_NUMBER() OVER (PARTITION BY truck_id ORDER BY sentiment_score ASC) as worst_review_rank
            FROM data.truck_review_sentiments
            WHERE sentiment_class = 'negative'
            AND sentiment_score < -0.8
        )
        SELECT 
            truck_id,
            review,
            sentiment_score
        FROM negative_reviews
        WHERE worst_review_rank = 1
        ORDER BY sentiment_score ASC
        LIMIT 3 -- top 3 only
      )
    )
  THEN
    BEGIN
        -- TODO add event
        LET rs RESULTSET := (
            WITH REVIEW_DATA AS (
                    SELECT truck_id, review
                    FROM TABLE(RESULT_SCAN(SNOWFLAKE.ALERT.GET_CONDITION_QUERY_UUID()))
                ),
                SUMMARIZED_CONTENT AS (
                SELECT 
                    SNOWFLAKE.CORTEX.COMPLETE(
                        'llama3.1-405b',
                        CONCAT(
                            'Summarize the review as bullets formatted for slack notification blocks with right and consistent emojis and always add truck id to the Review Alert header along with truck emoji and stay consistent with Header like <alert emoji> Review  <alert emoji> <truck emoji> <space> Truck ID - <truck id>:',
                            '<REVIEW>', 
                            REVIEW, 
                            '</REVIEW>',
                            'Quote the truck id.', 
                            TRUCK_ID,
                            '.Generate only Slack blocks and strictly ignore other text.'
                        )) AS SUMMARY
                FROM REVIEW_DATA
            ),
            FORMATTED_BLOCKS AS (
                SELECT SNOWFLAKE.NOTIFICATION.SANITIZE_WEBHOOK_CONTENT(SUMMARY) AS CLEAN_BLOCKS
                FROM SUMMARIZED_CONTENT
            ),
            JSON_BLOCKS AS (
                SELECT SNOWFLAKE.NOTIFICATION.APPLICATION_JSON(CONCAT('{"blocks":',CLEAN_BLOCKS,'}')) AS BLOCKS
                FROM FORMATTED_BLOCKS
            )
            -- slack message content blocks
            SELECT BLOCKS FROM JSON_BLOCKS
        );
    
        FOR record IN rs DO
            let slack_message varchar := record.BLOCKS;
            SYSTEM$LOG_INFO('SLACK MESSAGE:',OBJECT_CONSTRUCT('slack_message', slack_message));
            CALL SYSTEM$SEND_SNOWFLAKE_NOTIFICATION(
                :slack_message,
                SNOWFLAKE.NOTIFICATION.INTEGRATION('{{__slack_notification}}')
            );
        END FOR;
    END;


Let's trigger the alert immediately.

In [ ]:
EXECUTE ALERT {{__database}}.{{__alerts_schema}}.{{__truck_negatives_alert}};

Suspend the alert if needed.

In [ ]:
ALTER ALERT {{__database}}.{{__alerts_schema}}.{{__truck_negatives_alert}} SUSPEND;

## Alert and Notification History

Snowflake provides dedicated stored procedures to view the execution history of alerts and notifications. These procedures allow you to monitor and audit your alert and notification activities.

To retrieve historical data, use these stored procedures:

### Alert History
```sql
INFORMATION_SCHEMA.ALERT_HISTORY
```
This procedure returns detailed records of past alert executions.

### Notification History
```sql
INFORMATION_SCHEMA.NOTIFICATION_HISTORY
```

In [ ]:
st.header("Alert History")
scheduled_time_range_start = st.slider("Schedule Time Range Start(mins):",min_value=5,max_value=60)
#alert_history_tf=session.table_function(information_schema.alert_history)


In [ ]:
df=session.sql(f"""
Select name,database_name,schema_name,action,state,sql_error_message
from
  table(information_schema.alert_history(
    scheduled_time_range_start
      =>dateadd('minutes',-{scheduled_time_range_start},current_timestamp())))
order by scheduled_time desc
""")
st.dataframe(df)

In [ ]:
st.header("Notification History")
_start_time = st.slider("Start time(mins):",min_value=5,max_value=60)
#alert_history_tf=session.table_function(information_schema.alert_history)

In [ ]:
notify_df=session.sql(f"""
SELECT INTEGRATION_NAME,STATUS,ERROR_MESSAGE 
FROM TABLE(INFORMATION_SCHEMA.NOTIFICATION_HISTORY(
  START_TIME => dateadd('minutes',-{_start_time},current_timestamp()),
  INTEGRATION_NAME => '{__slack_notification}'
))
""")
st.dataframe(notify_df)

## Resource Cleanup

To prevent unnecessary resource consumption and cost.

In [ ]:
DROP NOTIFICATION INTEGRATION {{__slack_notification}};
DROP DATABASE {{__database}}